In [4]:
# Programming evacuation systems using Crowd Simulation

# Loading the pygame package
import pygame
import import_ipynb

# Importing locals
from pygame.locals import *
from additional_functions import *
from environment_3F import *
from agent import *

# Other packages
import sys
import numpy as np
import numpy.random as random
import math
import time


# data_matrix = np.loadtxt('room1_r', dtype=float) # Enable after first run
data_matrix = np.zeros((nr_experiments * nr_agents, 4))  # Delete/comment after first run

# Making sure we can run experiments one by one
j = 0  # add one after running

# Checking if all are executed
if j == nr_agents:
    print("nr of experiments reached")
    sys.exit()

# Initializing Pygame and font
pygame.init()
pygame.font.init()
timefont = pygame.font.SysFont('Comic Sans MS', 30)

""" 

Creating a screen with a room that is smaller than then screen 

"""

# Size of the screen
width = 1500
height = 800
size = width, height  # Do not adjust this

# Creating screen
roomscreen = pygame.display.set_mode(size)

# Making background white and creating colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
background_color = WHITE
roomscreen.fill(background_color)
pygame.display.update()

# Defining clock
clock = pygame.time.Clock()

# Move world view
world_offset = [0,0]

def main():
    # Now to let multiple objects move to the door we define
    # nr_agents = nr_agents
    agent_color = GREEN
    line_color = BLACK

    # initialize agents
    agents = []

    def positions(agents):
        for i in range(nr_agents):
            agent = Agent()
            agent.walls = walls
            agent.x = positionmatrix[j * nr_agents + i][0]
            agent.y = positionmatrix[j * nr_agents + i][1]
            agent.pos = np.array([agent.x, agent.y])
            agent.radius = positionmatrix[j * nr_agents + i][2]
            agent.mass = positionmatrix[j * nr_agents + i][3]
            agent.dSpeed = positionmatrix[j * nr_agents + i][4]
            agents.append(agent)

    positions(agents)

    count = 0
    start_time = time.time()
    run = True

    while run:

        # Updating time
        if count < nr_agents:
            current_time = time.time()
            elapsed_time = current_time - start_time
        else:
            for agent_i in agents:
                data_matrix[(j + 1) * nr_agents - 2][0] = elapsed_time
                data_matrix[(j + 1) * nr_agents - 1][0] = elapsed_time
                agents.remove(agent_i)
            for k in range(j * nr_agents, (j + 1) * nr_agents):
                data_matrix[k][1] = elapsed_time

        # Finding delta t for this frame
        dt = clock.tick(70) / 1000

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                (x, y) = pygame.mouse.get_pos()
                print(x, y)

        global world_offset

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            world_offset[0] = world_offset[0] + 30
        if keys[pygame.K_RIGHT]:
            world_offset[0] = world_offset[0] - 30
        if keys[pygame.K_UP]:
            world_offset[1] = world_offset[1] + 30
        if keys[pygame.K_DOWN]:
            world_offset[1] = world_offset[1] - 30

        roomscreen.fill(background_color)

        # draw walls
        for wall in walls:
            start_posw = np.array([wall[0], wall[1]])
            end_posw = np.array([wall[2], wall[3]])
            start_posx = start_posw
            end_posx = end_posw
            pygame.draw.line(roomscreen, line_color, start_posx + world_offset, end_posx + world_offset, 3)

        for agent_i in agents:
            agent_i.update_dest()
            agent_i.direction = normalize(agent_i.dest - agent_i.pos)
            agent_i.dVelocity = agent_i.dSpeed * agent_i.direction
            aVelocity_force = agent_i.velocity_force()
            people_interaction = 0.0
            wall_interaction = 0.0

            for agent_j in agents:
                if agent_i == agent_j: continue
                people_interaction += agent_i.f_ij(agent_j)

            for wall in walls:
                wall_interaction += agent_i.f_ik_wall(wall)

            sumForce = aVelocity_force + people_interaction + wall_interaction
            dv_dt = sumForce / agent_i.mass
            agent_i.aVelocity = agent_i.aVelocity + dv_dt * dt
            agent_i.pos = agent_i.pos + agent_i.aVelocity * dt

            # Avoiding disappearing agents
            if agent_i.pos[0] > 2900 or agent_i.pos[0] < 50 or agent_i.pos[1] > 2300 or agent_i.pos[1] < 50:
                main()
                sys.exit()

        for agent_i in agents:

            agent_i.time += clock.get_time() / 1000
            start_position = [0, 0]
            start_position[0] = int(agent_i.pos[0]) + world_offset[0]
            start_position[1] = int(agent_i.pos[1]) + world_offset[1]

            end_position = [0, 0]
            end_position[0] = int(agent_i.pos[0] + agent_i.aVelocity[0]) + world_offset[0]
            end_position[1] = int(agent_i.pos[1] + agent_i.aVelocity[1]) + world_offset[1]

            end_positionDV = [0, 0]
            end_positionDV[0] = int(agent_i.pos[0] + agent_i.dVelocity[0]) + world_offset[0]
            end_positionDV[1] = int(agent_i.pos[1] + agent_i.dVelocity[1]) + world_offset[1]

            if ((start_position[0] >= 1156.3/2 + 4203/2 + buffer + world_offset[0]) or (start_position[1] >= 751.1/2 + (3 * 885.8/2) + 321.6/2 + buffer + world_offset[1] and start_position[0] <= buffer + 735/2 + 324/2 + world_offset[0]) ) and agent_i.Goal == 0:
                agent_i.Goal = 1
                data_matrix[count + j * nr_agents][0] = agent_i.time
                # print('Time to Reach the Goal:', agent_i.time)

            if (start_position[0] >= 1156.3/2 + 4203/2 + buffer + world_offset[0]) or (start_position[1] >= 751.1/2 + (3 * 885.8/2) + 321.6/2 + buffer + world_offset[1] and start_position[0] <= buffer + 735/2 + 324/2 + world_offset[0]):
                data_matrix[count + j * nr_agents][2] = count
                data_matrix[count + j * nr_agents][3] = agent_i.countcollision
                count += 1
                agents.remove(agent_i)

            pygame.draw.circle(roomscreen, agent_color, start_position, round(agent_i.radius), 3)
            pygame.draw.line(roomscreen, agent_color, start_position, end_positionDV, 2)
        # visibility
        # pygame.draw.circle(roomscreen, RED, [700,400], 300, 1)

        pygame.draw.line(roomscreen, [255, 60, 0], start_position, end_positionDV, 2)

        # Present text on screen
        timestr = "Time: " + str(elapsed_time)
        agentstr = "Number of Agents Escaped: " + str(count) + "/" + str(nr_agents)
        timesurface = timefont.render(timestr, False, (0, 0, 0))
        agentsurface = timefont.render(agentstr, False, (0, 0, 0))
        roomscreen.blit(timesurface, (0, 0))
        roomscreen.blit(agentsurface, (0, 40))
        # Update the screen
        pygame.display.flip()

    pygame.quit()


main()
# np.savetxt('room1_vis', data_matrix)


importing Jupyter notebook from environment_3F.ipynb
X and Y Position: [348.20169609 140.41453292]
self.direction: [0.58437548 0.81148339]
X and Y Position: [1021.13161883  120.11939543]
self.direction: [0.08557066 0.9963321 ]
X and Y Position: [700.18029642 171.83753316]
self.direction: [0.41213682 0.91112197]
X and Y Position: [242.89287192 173.53076375]
self.direction: [0.84029186 0.5421343 ]
X and Y Position: [535.42876937 407.05131772]
self.direction: [0.07838859 0.99692288]
X and Y Position: [ 851.40333633 1425.75917812]
self.direction: [0.56955084 0.8219561 ]
X and Y Position: [692.41804548 559.24208893]
self.direction: [0.74198434 0.67041722]
X and Y Position: [1116.67589744  873.12823356]
self.direction: [-0.45284942  0.89158701]
X and Y Position: [ 565.52403195 1414.35632763]
self.direction: [0.94310763 0.33248759]
X and Y Position: [ 278.62796098 1479.42880142]
self.direction: [ 0.99768719 -0.06797255]
X and Y Position: [ 307.07582653 1239.731536  ]
self.direction: [ 0.87917

X and Y Position: [ 860.83630078 1554.78714346]
self.direction: [-0.20789647 -0.97815084]
X and Y Position: [192.34766744 570.43936745]
self.direction: [-0.01586452  0.99987415]
X and Y Position: [427.10775184 827.93003923]
self.direction: [0.5205009  0.85386112]
X and Y Position: [933.45416692 580.63341296]
self.direction: [0.3428448  0.93939206]
X and Y Position: [765.0420055  189.51543917]
self.direction: [-0.30191654  0.95333436]
X and Y Position: [510.20832272 450.93720424]
self.direction: [0.32022381 0.94734192]
X and Y Position: [590.90194166 536.57631311]
self.direction: [0.41645381 0.90915688]
X and Y Position: [930.23949816 458.13055344]
self.direction: [-0.23098054  0.97295837]
X and Y Position: [408.42308952 922.97660452]
self.direction: [0.71061563 0.70358043]
X and Y Position: [ 199.68048588 1289.48914596]
self.direction: [ 0.68654769 -0.72708477]
X and Y Position: [1124.38502757 1835.22689327]
self.direction: [-0.09018465 -0.99592506]
X and Y Position: [497.9877109  824.

X and Y Position: [547.15923997 430.19511717]
self.direction: [-0.14385018  0.98959948]
X and Y Position: [ 161.31176094 1060.0290887 ]
self.direction: [0.9904334  0.13799157]
X and Y Position: [190.54670995 173.75268702]
self.direction: [0.6989414  0.71517894]
X and Y Position: [445.38026071 604.0885402 ]
self.direction: [0.4099595  0.91210373]
X and Y Position: [ 273.91620847 1165.53614988]
self.direction: [ 0.46411998 -0.88577234]
X and Y Position: [ 217.96877154 1712.4988701 ]
self.direction: [ 0.343299   -0.93922617]
X and Y Position: [317.91207177 949.64665407]
self.direction: [-0.02967326  0.99955965]
X and Y Position: [ 366.32866237 1801.27803367]
self.direction: [ 0.88525728 -0.46510165]
X and Y Position: [ 299.54077702 1137.95032497]
self.direction: [0.8254204  0.56451852]
X and Y Position: [1117.62871262 1160.49941281]
self.direction: [-0.99975339 -0.02220719]
X and Y Position: [778.44706042 840.08764246]
self.direction: [0.25317729 0.9674199 ]
X and Y Position: [ 657.972448

X and Y Position: [ 997.70011705 1304.47240559]
self.direction: [-0.69485748 -0.71914747]
X and Y Position: [1084.5382036   833.91489366]
self.direction: [-0.73024655  0.6831837 ]
X and Y Position: [ 654.35340933 1116.74292089]
self.direction: [-0.60451785  0.79659159]
X and Y Position: [ 292.18119305 1634.81291005]
self.direction: [ 0.77007058 -0.6379587 ]
X and Y Position: [1011.12239788 1238.87761323]
self.direction: [0.98160399 0.19092828]
X and Y Position: [ 780.55976937 1370.42743749]
self.direction: [0.70394254 0.71025692]
X and Y Position: [619.72208166 826.06306204]
self.direction: [0.95622717 0.29262535]
X and Y Position: [646.54586559 827.91389588]
self.direction: [0.12554299 0.99208818]
X and Y Position: [213.91335017 680.59886128]
self.direction: [0.80781405 0.58943741]
X and Y Position: [1020.11081077  391.67251253]
self.direction: [-0.49328927  0.86986533]
X and Y Position: [ 893.54411355 1849.58127398]
self.direction: [ 0.08356088 -0.99650267]
X and Y Position: [641.185

X and Y Position: [580.83460624 331.80415136]
self.direction: [-0.30421313  0.95260399]
X and Y Position: [430.39291431 695.07362705]
self.direction: [-0.08285179  0.99656188]
X and Y Position: [ 553.93423775 1073.22488095]
self.direction: [0.70999339 0.70420834]
X and Y Position: [856.87724938 744.91069448]
self.direction: [0.26872707 0.96321636]
X and Y Position: [1049.03571087 1816.72066586]
self.direction: [-0.00233159 -0.99999728]
X and Y Position: [605.84534081 224.54974139]
self.direction: [0.46557177 0.88501013]
X and Y Position: [ 280.68730708 1677.26512976]
self.direction: [0.94436636 0.3288954 ]
X and Y Position: [622.69199454 424.02049577]
self.direction: [0.76185019 0.64775327]
X and Y Position: [1104.43860096  883.85776575]
self.direction: [-0.93052527 -0.36622769]
X and Y Position: [158.1622558  802.55881771]
self.direction: [0.50584637 0.86262359]
X and Y Position: [ 666.61618902 1098.27478273]
self.direction: [0.72422399 0.6895648 ]
X and Y Position: [402.92449555 901.

...

X and Y Position: [ 721.65200981 1068.70811867]
self.direction: [-0.82366283  0.56707984]
X and Y Position: [746.74330858 750.33758839]
self.direction: [0.18655355 0.98244479]
X and Y Position: [1046.67517595  349.70858813]
self.direction: [0.0983068  0.99515616]
X and Y Position: [1026.71243774 1141.59436264]
self.direction: [0.2550034  0.96694016]
X and Y Position: [ 899.21039569 1316.56372524]
self.direction: [-0.8522576  -0.52312234]
X and Y Position: [766.51404873 931.93365494]
self.direction: [0.15826658 0.98739642]
X and Y Position: [ 543.53799896 1778.30905918]
self.direction: [0.94553713 0.32551425]
X and Y Position: [ 328.18898747 1151.0617805 ]
self.direction: [0.98569528 0.16853731]
X and Y Position: [ 414.83362442 1291.14424818]
self.direction: [0.08826203 0.99609729]
X and Y Position: [ 328.3403186 1086.1916743]
self.direction: [ 0.93684043 -0.34975708]
X and Y Position: [164.18191824 893.45053041]
self.direction: [0.9994857 0.0320676]
X and Y Position: [ 661.92030972 129

X and Y Position: [1143.83004263 1754.03465303]
self.direction: [-0.70511029 -0.70909765]
X and Y Position: [ 256.6187958  1379.67158392]
self.direction: [0.82135418 0.57041854]
X and Y Position: [1173.98904392  224.73666012]
self.direction: [-0.53672461  0.84375749]
X and Y Position: [1110.93996738  573.43869315]
self.direction: [0.06081506 0.99814905]
X and Y Position: [ 255.75951787 1636.37654812]
self.direction: [ 0.55883675 -0.82927769]
X and Y Position: [963.97740223 534.53778589]
self.direction: [-0.65640921  0.75440503]
X and Y Position: [1027.68292328 1133.72416624]
self.direction: [-0.99302612 -0.11789453]
X and Y Position: [672.04688946 546.5925295 ]
self.direction: [-0.56140228  0.82754304]
X and Y Position: [ 436.45397337 1625.08996603]
self.direction: [0.93821135 0.34606279]
X and Y Position: [772.51853575 131.17072282]
self.direction: [0.10691968 0.99426766]
X and Y Position: [1159.39080706 1219.06953706]
self.direction: [-0.97932801  0.20227866]
X and Y Position: [620.2

X and Y Position: [1024.28601583  774.9597122 ]
self.direction: [0.08689222 0.99621772]
X and Y Position: [ 160.49949317 1825.61489568]
self.direction: [ 0.64006547 -0.76832037]
X and Y Position: [ 610.97413186 1408.57757976]
self.direction: [0.8685881  0.49553477]
X and Y Position: [ 597.3500345  1502.87547226]
self.direction: [-0.83412828  0.55157049]
X and Y Position: [ 495.3321428  1831.82525679]
self.direction: [-0.47734029 -0.87871853]
X and Y Position: [489.10119675 210.2365733 ]
self.direction: [0.78052013 0.62513065]
X and Y Position: [ 482.4549994  1709.73205371]
self.direction: [-0.95211201  0.30574943]
X and Y Position: [1156.79873474  801.89257124]
self.direction: [-0.25186752  0.96776172]
X and Y Position: [ 138.18909948 1184.97954598]
self.direction: [0.8736463  0.48656154]
X and Y Position: [363.32213607 568.98148089]
self.direction: [0.7517252  0.65947648]
X and Y Position: [345.9850451  955.37666875]
self.direction: [-0.50310042 -0.86422796]
X and Y Position: [ 860.22

X and Y Position: [1073.21419189  228.449128  ]
self.direction: [-0.43241496  0.90167472]
X and Y Position: [291.48225383 490.25845992]
self.direction: [0.07851927 0.9969126 ]
X and Y Position: [172.26332387 903.01974422]
self.direction: [ 0.98006312 -0.19868638]
X and Y Position: [ 289.43059777 1205.76615059]
self.direction: [0.92563189 0.37842517]
X and Y Position: [528.85419558 539.18360575]
self.direction: [-0.17288732  0.98494161]
X and Y Position: [560.94579714 581.05824525]
self.direction: [0.37649533 0.92641851]
X and Y Position: [1032.07337576 1063.78799748]
self.direction: [-0.98413145  0.17744093]
X and Y Position: [322.57099372 285.58769933]
self.direction: [-0.39744516  0.91762593]
X and Y Position: [593.8269632  737.31226751]
self.direction: [0.0666691  0.99777514]
X and Y Position: [1150.71839411 1586.35501806]
self.direction: [-0.58831522 -0.80863169]
X and Y Position: [944.08911247 625.66788632]
self.direction: [-0.29165707  0.95652295]
X and Y Position: [ 337.66507335

SystemExit: 